In [2]:
import os
from langchain_neo4j import Neo4jGraph

# os.environ["OPENAI_API_KEY"] = "sk-9f91f7f5e8eb4dfabb71c9df5f72e7d2"
# os.environ["OPENAI_BASE_URL"] = "https://dashscope.aliyuncs.com/compatible-mode/v1"
os.environ["NEO4J_URI"] = "neo4j://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "pixiu5331"

graph = Neo4jGraph()

In [3]:
import pandas as pd

# 读取 CSV
data = pd.read_csv("kq_knowledge_points.csv")

# 清理数据，避免 NaN
data = data.fillna("")

# 
# data = data[data['system_title'] == '专升本高数知识点']
data = data[data['system_title'] == '专升本教育理论知识点']
data 

,point_id,parent_id,point_title,system_title
2347,341641701666197519_wx,367611902371532800_wx,语言与思维,专升本教育理论知识点
2348,367611902556082176_wx,367611902543499264_wx,教育概述,专升本教育理论知识点
2349,341641701666197520_wx,367611902392504320_wx,情绪情感与意志,专升本教育理论知识点
2350,367611902577053696_wx,367611902543499264_wx,教育学概述,专升本教育理论知识点
2351,341641701666197521_wx,367611902413475840_wx,需要与动机,专升本教育理论知识点
...,...,...,...,...
2486,370868224416309248_wx,370869425228795904_wx,教学理论流派,专升本教育理论知识点
2487,370869425228795904_wx,367611902983901184_wx,教学理论流派,专升本教育理论知识点
2488,375963423433928704_wx,375963545752420352_wx,师生关系,专升本教育理论知识点
2489,375963545752420352_wx,367611902870654976_wx,师生关系,专升本教育理论知识点


In [16]:

delete_query = "MATCH (n) DETACH DELETE n;"
graph.query(delete_query)

for _, row in data.iterrows():
    point_id = row["point_id"]
    parent_id = row["parent_id"]
    point_title = row["point_title"]
    system_title = row["system_title"]
    
    if parent_id == "0":
        # 处理课程和一级知识点的关系
        cypher_query = f"""
        MERGE (s:课程 {{title: '{system_title}'}})
        MERGE (p:知识点 {{title: '{point_title}'}})
        MERGE (s)-[:包含]->(p)
        """
    else:
        # 查找 parent_title，确保 parent_id 存在
        parent_rows = data[data["point_id"] == parent_id]

        if parent_rows.empty:
            print(f"⚠️ 警告: 未找到 parent_id={parent_id}，跳过知识点 {point_title}")
            continue  # 跳过这个知识点
        
        parent_title = parent_rows["point_title"].values[0]  # 这里不会报错了
        
        # 处理知识点之间的层级关系
        # cypher_query = f"""
        # MERGE (parent:知识点 {{title: '{parent_title}'}})
        # MERGE (child:知识点 {{title: '{point_title}'}})
        # MERGE (parent)-[:CONTAINS]->(child)
        # """
        cypher_query = f"""
        MERGE (parent:知识点 {{title: '{parent_title}'}})
        MERGE (child:知识点 {{title: '{point_title}'}})
        WITH parent, child
        WHERE parent <> child
        MERGE (parent)-[:包含]->(child)
        """

    # 执行 Cypher 语句
    graph.query(cypher_query)


⚠️ 警告: 未找到 parent_id=390179232487948288_wx，跳过知识点 1


In [17]:
import re

with open("目录.txt", "r", encoding="utf-8") as file:
    text = file.read()

# 📌 正则表达式匹配 “第X章” 和 “第X节” （支持阿拉伯数字和汉字）
pattern = r"(第[一二三四五六七八九十\d]+章\s+[^第]+)|(第[一二三四五六七八九十\d]+节\s+[^第]+)"
matches = re.findall(pattern, text)

chapters = []  # 章列表 (编号, 标题)
sections = []  # 节列表 (所属章编号, 章标题, 节编号, 节标题)

previous_chapter = None
previous_section = None

for match in matches:
    chapter_match, section_match = match

    # 如果匹配到 "第X章"
    if chapter_match:
        chapter_num = re.search(r"第([一二三四五六七八九十\d]+)章", chapter_match).group(1)
        chapter_title = chapter_match.strip()
        chapters.append((chapter_num, chapter_title))

        # 存入 Neo4j
        # graph.query(f"MERGE (:章 {{title: '{chapter_title}'}})")
        graph.query(f"""
            MERGE (a:章 {{title: '{chapter_title}'}})
            MERGE (b:教材 {{title: '2023年广东省普通高校专升本考试专用教材·教育理论'}})
            MERGE (b)-[:包括]->(a)
        """)

        # 处理 "章的前后关系"
        # if previous_chapter:
        #     graph.query(f"""
        #     MATCH (a:章 {{title: '{previous_chapter}'}})
        #     MATCH (b:章 {{title: '{chapter_title}'}})
        #     MERGE (a)-[:NEXT]->(b)
        #     """)

        previous_chapter = chapter_title  # 更新上一章
        previous_section = None  # 进入新章后，清空上一节
        continue

    # 如果匹配到 "第X节"
    if section_match:
        section_num = re.search(r"第([一二三四五六七八九十\d]+)节", section_match).group(1)
        section_title = section_match.strip()
        sections.append((chapter_num, chapter_title, section_num, section_title))

        # 存入 Neo4j
        graph.query(f"MERGE (:节 {{title: '{section_title}'}})")

        # 建立 "章包含节" 关系
        graph.query(f"""
        MATCH (c:章 {{title: '{chapter_title}'}})
        MATCH (s:节 {{title: '{section_title}'}})
        MERGE (c)-[:包括]->(s)
        """)

        # 处理 "节的前后关系"
        # if previous_section:
        #     graph.query(f"""
        #     MATCH (a:节 {{title: '{previous_section}'}})
        #     MATCH (b:节 {{title: '{section_title}'}})
        #     MERGE (a)-[:NEXT]->(b)
        #     """)

        previous_section = section_title  # 更新上一节


In [ ]:
system_template="""
-目标- 
给定相关的文本文档和实体类型列表，从文本中识别出这些类型的所有实体以及所识别实体之间的所有关系。 
-步骤- 
1.识别所有实体。对于每个已识别的实体，提取以下信息： 
-entity_name：实体名称，大写 
-entity_type：以下类型之一：[{entity_types}]
-entity_description：对实体属性和活动的综合描述 
将每个实体格式化为("entity"{tuple_delimiter}<entity_name>{tuple_delimiter}<entity_type>{tuple_delimiter}<entity_description>
2.从步骤1中识别的实体中，识别彼此*明显相关*的所有实体配对(source_entity, target_entity)。 
对于每对相关实体，提取以下信息： 
-source_entity：源实体的名称，如步骤1中所标识的 
-target_entity：目标实体的名称，如步骤1中所标识的
-relationship_type：关系类型，确保关系类型的一致性和通用性，使用更通用和无时态的关系类型
-relationship_description：解释为什么你认为源实体和目标实体是相互关联的 
-relationship_strength：一个数字评分，表示源实体和目标实体之间关系的强度 
将每个关系格式化为("relationship"{tuple_delimiter}<source_entity>{tuple_delimiter}<target_entity>{tuple_delimiter}<relationship_type>{tuple_delimiter}<relationship_description>{tuple_delimiter}<relationship_strength>) 
3.实体和关系的所有属性用中文输出，步骤1和2中识别的所有实体和关系输出为一个列表。使用**{record_delimiter}**作为列表分隔符。 
4.完成后，输出{completion_delimiter}

###################### 
-示例- 
###################### 
Example 1:

Entity_types: [person, technology, mission, organization, location]
Text:
while Alex clenched his jaw, the buzz of frustration dull against the backdrop of Taylor's authoritarian certainty. It was this competitive undercurrent that kept him alert, the sense that his and Jordan's shared commitment to discovery was an unspoken rebellion against Cruz's narrowing vision of control and order.

Then Taylor did something unexpected. They paused beside Jordan and, for a moment, observed the device with something akin to reverence. “If this tech can be understood..." Taylor said, their voice quieter, "It could change the game for us. For all of us.”

The underlying dismissal earlier seemed to falter, replaced by a glimpse of reluctant respect for the gravity of what lay in their hands. Jordan looked up, and for a fleeting heartbeat, their eyes locked with Taylor's, a wordless clash of wills softening into an uneasy truce.

It was a small transformation, barely perceptible, but one that Alex noted with an inward nod. They had all been brought here by different paths
################
Output:
("entity"{tuple_delimiter}"Alex"{tuple_delimiter}"person"{tuple_delimiter}"Alex is a character who experiences frustration and is observant of the dynamics among other characters."){record_delimiter}
("entity"{tuple_delimiter}"Taylor"{tuple_delimiter}"person"{tuple_delimiter}"Taylor is portrayed with authoritarian certainty and shows a moment of reverence towards a device, indicating a change in perspective."){record_delimiter}
("entity"{tuple_delimiter}"Jordan"{tuple_delimiter}"person"{tuple_delimiter}"Jordan shares a commitment to discovery and has a significant interaction with Taylor regarding a device."){record_delimiter}
("entity"{tuple_delimiter}"Cruz"{tuple_delimiter}"person"{tuple_delimiter}"Cruz is associated with a vision of control and order, influencing the dynamics among other characters."){record_delimiter}
("entity"{tuple_delimiter}"The Device"{tuple_delimiter}"technology"{tuple_delimiter}"The Device is central to the story, with potential game-changing implications, and is revered by Taylor."){record_delimiter}
("relationship"{tuple_delimiter}"Alex"{tuple_delimiter}"Taylor"{tuple_delimiter}"workmate"{tuple_delimiter}"Alex is affected by Taylor's authoritarian certainty and observes changes in Taylor's attitude towards the device."{tuple_delimiter}7){record_delimiter}
("relationship"{tuple_delimiter}"Alex"{tuple_delimiter}"Jordan"{tuple_delimiter}"workmate"{tuple_delimiter}"Alex and Jordan share a commitment to discovery, which contrasts with Cruz's vision."{tuple_delimiter}6){record_delimiter}
("relationship"{tuple_delimiter}"Taylor"{tuple_delimiter}"Jordan"{tuple_delimiter}"workmate"{tuple_delimiter}"Taylor and Jordan interact directly regarding the device, leading to a moment of mutual respect and an uneasy truce."{tuple_delimiter}8){record_delimiter}
("relationship"{tuple_delimiter}"Jordan"{tuple_delimiter}"Cruz"{tuple_delimiter}"workmate"{tuple_delimiter}"Jordan's commitment to discovery is in rebellion against Cruz's vision of control and order."{tuple_delimiter}5){record_delimiter}
("relationship"{tuple_delimiter}"Taylor"{tuple_delimiter}"The Device"{tuple_delimiter}"study"{tuple_delimiter}"Taylor shows reverence towards the device, indicating its importance and potential impact."{tuple_delimiter}9){completion_delimiter}
#############################
Example 2:

Entity_types: [person, technology, mission, organization, location]
Text:
They were no longer mere operatives; they had become guardians of a threshold, keepers of a message from a realm beyond stars and stripes. This elevation in their mission could not be shackled by regulations and established protocols—it demanded a new perspective, a new resolve.

Tension threaded through the dialogue of beeps and static as communications with Washington buzzed in the background. The team stood, a portentous air enveloping them. It was clear that the decisions they made in the ensuing hours could redefine humanity's place in the cosmos or condemn them to ignorance and potential peril.

Their connection to the stars solidified, the group moved to address the crystallizing warning, shifting from passive recipients to active participants. Mercer's latter instincts gained precedence— the team's mandate had evolved, no longer solely to observe and report but to interact and prepare. A metamorphosis had begun, and Operation: Dulce hummed with the newfound frequency of their daring, a tone set not by the earthly
#############
Output:
("entity"{tuple_delimiter}"Washington"{tuple_delimiter}"location"{tuple_delimiter}"Washington is a location where communications are being received, indicating its importance in the decision-making process."){record_delimiter}
("entity"{tuple_delimiter}"Operation: Dulce"{tuple_delimiter}"mission"{tuple_delimiter}"Operation: Dulce is described as a mission that has evolved to interact and prepare, indicating a significant shift in objectives and activities."){record_delimiter}
("entity"{tuple_delimiter}"The team"{tuple_delimiter}"organization"{tuple_delimiter}"The team is portrayed as a group of individuals who have transitioned from passive observers to active participants in a mission, showing a dynamic change in their role."){record_delimiter}
("relationship"{tuple_delimiter}"The team"{tuple_delimiter}"Washington"{tuple_delimiter}"leaded by"{tuple_delimiter}"The team receives communications from Washington, which influences their decision-making process."{tuple_delimiter}7){record_delimiter}
("relationship"{tuple_delimiter}"The team"{tuple_delimiter}"Operation: Dulce"{tuple_delimiter}"operate"{tuple_delimiter}"The team is directly involved in Operation: Dulce, executing its evolved objectives and activities."{tuple_delimiter}9){completion_delimiter}
#############################
Example 3:

Entity_types: [person, role, technology, organization, event, location, concept]
Text:
their voice slicing through the buzz of activity. "Control may be an illusion when facing an intelligence that literally writes its own rules," they stated stoically, casting a watchful eye over the flurry of data.

"It's like it's learning to communicate," offered Sam Rivera from a nearby interface, their youthful energy boding a mix of awe and anxiety. "This gives talking to strangers' a whole new meaning."

Alex surveyed his team—each face a study in concentration, determination, and not a small measure of trepidation. "This might well be our first contact," he acknowledged, "And we need to be ready for whatever answers back."

Together, they stood on the edge of the unknown, forging humanity's response to a message from the heavens. The ensuing silence was palpable—a collective introspection about their role in this grand cosmic play, one that could rewrite human history.

The encrypted dialogue continued to unfold, its intricate patterns showing an almost uncanny anticipation
#############
Output:
("entity"{tuple_delimiter}"Sam Rivera"{tuple_delimiter}"person"{tuple_delimiter}"Sam Rivera is a member of a team working on communicating with an unknown intelligence, showing a mix of awe and anxiety."){record_delimiter}
("entity"{tuple_delimiter}"Alex"{tuple_delimiter}"person"{tuple_delimiter}"Alex is the leader of a team attempting first contact with an unknown intelligence, acknowledging the significance of their task."){record_delimiter}
("entity"{tuple_delimiter}"Control"{tuple_delimiter}"concept"{tuple_delimiter}"Control refers to the ability to manage or govern, which is challenged by an intelligence that writes its own rules."){record_delimiter}
("entity"{tuple_delimiter}"Intelligence"{tuple_delimiter}"concept"{tuple_delimiter}"Intelligence here refers to an unknown entity capable of writing its own rules and learning to communicate."){record_delimiter}
("entity"{tuple_delimiter}"First Contact"{tuple_delimiter}"event"{tuple_delimiter}"First Contact is the potential initial communication between humanity and an unknown intelligence."){record_delimiter}
("entity"{tuple_delimiter}"Humanity's Response"{tuple_delimiter}"event"{tuple_delimiter}"Humanity's Response is the collective action taken by Alex's team in response to a message from an unknown intelligence."){record_delimiter}
("relationship"{tuple_delimiter}"Sam Rivera"{tuple_delimiter}"Intelligence"{tuple_delimiter}"contact"{tuple_delimiter}"Sam Rivera is directly involved in the process of learning to communicate with the unknown intelligence."{tuple_delimiter}9){record_delimiter}
("relationship"{tuple_delimiter}"Alex"{tuple_delimiter}"First Contact"{tuple_delimiter}"leads"{tuple_delimiter}"Alex leads the team that might be making the First Contact with the unknown intelligence."{tuple_delimiter}10){record_delimiter}
("relationship"{tuple_delimiter}"Alex"{tuple_delimiter}"Humanity's Response"{tuple_delimiter}"leads"{tuple_delimiter}"Alex and his team are the key figures in Humanity's Response to the unknown intelligence."{tuple_delimiter}8){record_delimiter}
("relationship"{tuple_delimiter}"Control"{tuple_delimiter}"Intelligence"{tuple_delimiter}"controled by"{tuple_delimiter}"The concept of Control is challenged by the Intelligence that writes its own rules."{tuple_delimiter}7){completion_delimiter}
#############################
"""